In [ ]:
# --- CELLA 1: SYSTEM ARCHITECTURE & DEFINITIONS ---
import sys
import os
import warnings
import numpy as np
from scipy.optimize import minimize as scipy_minimize
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from datetime import datetime
import time
import multiprocessing

# Dependency Check & Import
try:
    from pymoo.core.problem import ElementwiseProblem
    from pymoo.algorithms.moo.moead import MOEAD
    from pymoo.optimize import minimize
    from pymoo.util.ref_dirs import get_reference_directions
    from pymoo.termination import get_termination
except ImportError:
    raise ImportError("Critical Dependency Missing: 'pymoo' is required. Please install it.")

# ESA Problem Module Import
try:
    import constellations_udp as udp
except ImportError:
    print("Warning: 'constellations_udp.py' not found in the current directory.")
    print("Ensure the file exists and the 'data/spoc2/constellations' path is valid.")

# Configuration
warnings.filterwarnings('ignore')

# Detect available cores
N_CORES = min(10, multiprocessing.cpu_count() - 1)  # Use 10 cores or available-1
print(f"⚡ Detected {multiprocessing.cpu_count()} CPU cores")
print(f"⚡ Using {N_CORES} cores for parallel processing")

class SpOCConstrainedWrapper(ElementwiseProblem):
    """
    Pymoo Wrapper for the ESA SpOC Challenge Problem.
    Implements a Penalty Method to handle the native constraints of the UDP within the MOEA/D algorithm,
    which does not support explicit constraint handling in its standard implementation.
    """
    def __init__(self):
        self.esa_problem = udp.constellation_udp()
        lb, ub = self.esa_problem.get_bounds()
        # Initialize with n_ieq_constr=0 to bypass MOEA/D assertions.
        # Constraints are incorporated directly into the objective function (Penalty).
        super().__init__(n_var=20, n_obj=2, n_ieq_constr=0, xl=np.array(lb), xu=np.array(ub))
    
    def _evaluate(self, x, out, *args, **kwargs):
        # Integer variable handling via rounding (Gene indices 10-19)
        x_eval = x.copy()
        x_eval[10:20] = np.round(x_eval[10:20]).astype(int)
        
        try:
            # ESA Simulator Call
            f = self.esa_problem.fitness(list(x_eval))
            J1, J2 = f[0], f[1]  # Objectives
            c1, c2 = f[2], f[3]  # Constraints (<=0 is valid)
            
            # Adaptive Penalty Method Formulation
            # Dynamic penalty based on generation progress
            penalty_factor = 1e5  # Fixed large penalty for feasibility dominance
            
            penalty = 0.0
            if c1 > 0:
                penalty += penalty_factor + c1 * 1000
            if c2 > 0:
                penalty += penalty_factor + c2 * 1000
            
            # Return penalized objectives
            out["F"] = [J1 + penalty, J2 + penalty]
        except Exception as e:
            # Fallback for simulator crashes (e.g., SGP4 propagation errors)
            out["F"] = [1e9, 1e9]

def train_and_predict_surrogate_global(history_X, history_F, problem_inst, n_candidates=50000):
    """
    Phase 2a: Global Surrogate-Assisted Optimization.
    Uses Random Forest to learn the fitness landscape from MOEA/D history.
    """
    print(" > Initializing GLOBAL Surrogate Model (Random Forest)...")
    
    # Filter only valid or near-valid solutions for training stability
    valid_mask = np.all(history_F < 5000, axis=1)  # Slightly more relaxed for global
    
    if np.sum(valid_mask) < 50:  # Need more data for reliable training
        print(f" > Insufficient valid data points ({np.sum(valid_mask)}). Skipping Global Surrogate phase.")
        return []
    
    X_train = history_X[valid_mask]
    y_train = history_F[valid_mask]
    
    print(f" > Training on {len(X_train)} valid solutions...")
    
    # Enhanced Random Forest with more trees for better accuracy
    regr = RandomForestRegressor(
        n_estimators=200,  # More trees for better accuracy
        max_depth=30,      # Deeper trees for complex landscape
        min_samples_split=5,
        min_samples_leaf=2,
        max_features='sqrt',
        random_state=42,
        n_jobs=N_CORES,  # Use all available cores
        verbose=0
    )
    regr.fit(X_train, y_train)
    
    # Feature importance analysis
    feature_importance = regr.feature_importances_
    print(f" > Top 5 important features: {np.argsort(feature_importance)[-5:][::-1]}")
    
    # Candidate Generation - more diverse
    candidates = []
    n_parents = 50  # Use more parents for diversity
    
    # Select top parents based on weighted sum
    weighted_scores = 0.97 * y_train[:, 0] + 0.03 * y_train[:, 1]
    best_parents_idx = np.argsort(weighted_scores)[:n_parents]
    parents = X_train[best_parents_idx]
    
    iterations = n_candidates // len(parents)
    for _ in range(iterations):
        p = parents[np.random.choice(len(parents))]
        # Adaptive perturbation based on feature importance
        noise_scale = 0.05 * (1 + feature_importance)  # More perturbation on important features
        noise = np.random.normal(0, noise_scale, 20)
        child = np.clip(p + noise, problem_inst.xl, problem_inst.xu)
        candidates.append(child)
    
    # Add some purely random candidates for exploration
    n_random = n_candidates // 10
    for _ in range(n_random):
        random_candidate = np.random.uniform(problem_inst.xl, problem_inst.xu)
        candidates.append(random_candidate)
    
    candidates = np.array(candidates)
    
    # Surrogate Prediction - parallelized
    print(f" > Predicting {len(candidates)} candidates...")
    preds = regr.predict(candidates)
    
    # Selection Strategy: Pareto front approximation
    # First filter by predicted feasibility
    predicted_feasible = np.all(preds < 1000, axis=1)
    feasible_preds = preds[predicted_feasible]
    feasible_candidates = candidates[predicted_feasible]
    
    if len(feasible_preds) == 0:
        print(" > No feasible predictions. Using weighted sum on all predictions.")
        feasible_preds = preds
        feasible_candidates = candidates
    
    # Weighted sum for initial filtering
    weighted_scores = 0.97 * feasible_preds[:, 0] + 0.03 * feasible_preds[:, 1]
    top_k = min(50, len(feasible_preds))  # Take more for verification
    best_idxs = np.argsort(weighted_scores)[:top_k]
    
    verified_solutions = []
    print(f" > Verifying top {len(best_idxs)} candidates with physical simulator...")
    
    for i, idx in enumerate(best_idxs):
        sol = feasible_candidates[idx]
        # Integer constraint enforcement
        x_check = sol.copy()
        x_check[10:20] = np.round(x_check[10:20]).astype(int)
        
        try:
            fit = problem_inst.esa_problem.fitness(list(x_check))
            
            # Hard Constraint Check
            if fit[2] <= 0 and fit[3] <= 0:
                verified_solutions.append(sol)
        except:
            continue
        
        # Progress indicator
        if (i + 1) % 10 == 0:
            print(f"   Verified {i + 1}/{len(best_idxs)} candidates...")
    
    print(f" > Global Surrogate Phase yielded {len(verified_solutions)} valid improvements.")
    return verified_solutions

def train_and_predict_surrogate_local(best_solution, history_X, history_F, problem_inst, n_candidates=20000):
    """
    Phase 2b: Local Surrogate Refinement (Prof's suggestion).
    Re-trains surrogate in a local neighborhood of the best solution for fine-grained search.
    """
    print(" > Initializing LOCAL Surrogate Model (Focused Random Forest)...")
    
    # Define local neighborhood (2% of variable range)
    ranges = problem_inst.xu - problem_inst.xl
    initial_radius = 0.02 * ranges  # Very local: 2% of range
    
    # Filter solutions within the local neighborhood
    distances = np.linalg.norm(history_X - best_solution, axis=1)
    neighborhood_mask = distances < np.linalg.norm(initial_radius)
    
    # If insufficient points, gradually expand neighborhood
    expansion_factor = 1.0
    while np.sum(neighborhood_mask) < 100 and expansion_factor <= 5.0:
        expanded_radius = (0.02 * expansion_factor) * ranges
        neighborhood_mask = np.all(
            np.abs(history_X - best_solution) <= expanded_radius,
            axis=1
        )
        expansion_factor += 0.5
    
    print(f" > Found {np.sum(neighborhood_mask)} solutions in local neighborhood (radius: {expansion_factor*2:.1f}%)")
    
    if np.sum(neighborhood_mask) < 30:
        print(" > Insufficient points for local surrogate. Skipping.")
        return []
    
    X_local = history_X[neighborhood_mask]
    y_local = history_F[neighborhood_mask]
    
    # Train specialized local surrogate
    rf_local = RandomForestRegressor(
        n_estimators=300,  # More trees for local precision
        max_depth=20,      # Shallower trees to prevent overfitting
        min_samples_split=10,
        min_samples_leaf=5,
        max_features=0.7,  # Limit features for local model
        random_state=42,
        n_jobs=N_CORES,
        verbose=0
    )
    rf_local.fit(X_local, y_local)
    
    # Generate highly localized candidates
    candidates_local = []
    
    # Strategy 1: Small perturbations around best solution
    n_small_perturbations = n_candidates // 2
    for _ in range(n_small_perturbations):
        # Very small noise for fine-tuning
        noise_std = 0.005 * (problem_inst.xu - problem_inst.xl)  # 0.5% of range
        noise = np.random.normal(0, noise_std)
        child = np.clip(best_solution + noise, problem_inst.xl, problem_inst.xu)
        candidates_local.append(child)
    
    # Strategy 2: Interpolation between best solutions in neighborhood
    if len(X_local) > 5:
        n_interpolations = n_candidates // 4
        top_local_idx = np.argsort(0.97 * y_local[:, 0] + 0.03 * y_local[:, 1])[:5]
        top_solutions = X_local[top_local_idx]
        
        for _ in range(n_interpolations):
            # Random convex combination
            alpha = np.random.dirichlet(np.ones(len(top_solutions)))
            child = np.sum(alpha[:, np.newaxis] * top_solutions, axis=0)
            # Add tiny noise
            noise = np.random.normal(0, 0.001, 20)
            child = np.clip(child + noise, problem_inst.xl, problem_inst.xu)
            candidates_local.append(child)
    
    # Strategy 3: Gaussian Process guided sampling (optional)
    n_gp_samples = n_candidates // 4
    try:
        # Fit Gaussian Process for uncertainty estimation
        gp_kernel = Matern(nu=2.5)
        gp = GaussianProcessRegressor(kernel=gp_kernel, n_restarts_optimizer=5)
        
        # Scale outputs for GP
        y_mean = np.mean(y_local[:, :2], axis=0)
        y_std = np.std(y_local[:, :2], axis=0)
        y_scaled = (y_local[:, :2] - y_mean) / (y_std + 1e-8)
        
        gp.fit(X_local, y_scaled[:, 0])  # Fit on J1 only for speed
        
        # Use GP to sample promising regions
        for _ in range(n_gp_samples):
            # Latin hypercube sampling in local region
            sample = best_solution.copy()
            perturb_mask = np.random.rand(20) < 0.3  # Perturb only 30% of variables
            if np.any(perturb_mask):
                perturbation = np.random.normal(0, 0.01, 20)
                sample[perturb_mask] = np.clip(
                    sample[perturb_mask] + perturbation[perturb_mask],
                    problem_inst.xl[perturb_mask],
                    problem_inst.xu[perturb_mask]
                )
            candidates_local.append(sample)
    except:
        # Fallback to random perturbations if GP fails
        for _ in range(n_gp_samples):
            noise = np.random.normal(0, 0.01, 20)
            child = np.clip(best_solution + noise, problem_inst.xl, problem_inst.xu)
            candidates_local.append(child)
    
    candidates_local = np.array(candidates_local)
    
    # Predict with local surrogate
    print(f" > Predicting {len(candidates_local)} local candidates...")
    preds_local = rf_local.predict(candidates_local)
    
    # Very aggressive selection for local search
    weighted_scores = 0.97 * preds_local[:, 0] + 0.03 * preds_local[:, 1]
    top_local_k = min(20, len(candidates_local))
    best_local_idxs = np.argsort(weighted_scores)[:top_local_k]
    
    verified_local = []
    print(f" > Verifying top {len(best_local_idxs)} local candidates...")
    
    for idx in best_local_idxs:
        sol = candidates_local[idx]
        x_check = sol.copy()
        x_check[10:20] = np.round(x_check[10:20]).astype(int)
        
        try:
            fit = problem_inst.esa_problem.fitness(list(x_check))
            
            if fit[2] <= 0 and fit[3] <= 0:
                verified_local.append(sol)
        except:
            continue
    
    print(f" > Local Surrogate Refinement yielded {len(verified_local)} valid improvements.")
    return verified_local

def refine_solution_local_search(start_sol, problem_inst, max_iter=100):
    """
    Phase 3: Enhanced Targeted Local Polishing.
    Performs multi-start local optimization for robustness.
    """
    print(" > Starting Enhanced Local Search (Multi-start Nelder-Mead)...")
    
    def objective_wrapper(etas, x_base, fixed_integers):
        x = x_base.copy()
        # Update only Eta parameters
        x[4], x[9] = etas[0], etas[1]
        
        # Apply fixed integers
        x[10:20] = fixed_integers
        
        try:
            f = problem_inst.esa_problem.fitness(list(x))
            
            # Hard penalty for constraints during polishing
            if f[2] > 0 or f[3] > 0:
                return 1e9
            
            # Target Objective: Weighted Sum
            return 0.97 * f[0] + 0.03 * f[1]
        except:
            return 1e9
    
    # Fix integer variables from best solution
    fixed_integers = np.round(start_sol[10:20]).astype(int)
    
    # Multiple starting points for robustness
    start_points = [
        [start_sol[4], start_sol[9]],  # Original
        [start_sol[4] * 0.9, start_sol[9] * 0.9],
        [start_sol[4] * 1.1, start_sol[9] * 0.9],
        [start_sol[4] * 0.9, start_sol[9] * 1.1],
        [start_sol[4] * 1.1, start_sol[9] * 1.1],
    ]
    
    best_value = float('inf')
    best_etas = None
    
    for i, x0 in enumerate(start_points):
        print(f"   Local search attempt {i + 1}/{len(start_points)}...")
        
        try:
            res = scipy_minimize(
                lambda etas: objective_wrapper(etas, start_sol, fixed_integers),
                x0,
                method='Nelder-Mead',
                bounds=((1.0, 1000.0), (1.0, 1000.0)),
                options={
                    'maxiter': max_iter // len(start_points),
                    'xatol': 1e-6,
                    'fatol': 1e-6,
                    'adaptive': True
                }
            )
            
            if res.success and res.fun < best_value:
                best_value = res.fun
                best_etas = res.x
        except:
            continue
    
    refined_sol = start_sol.copy()
    if best_etas is not None:
        refined_sol[4], refined_sol[9] = best_etas[0], best_etas[1]
        print(f"   Local search improved score from {objective_wrapper([start_sol[4], start_sol[9]], start_sol, fixed_integers):.6f} to {best_value:.6f}")
    else:
        print("   Local search failed to improve solution.")
    
    return refined_sol

print("✅ System Architecture initialized successfully.")

In [ ]:
# --- CELLA 2: EXECUTION PIPELINE ---

# --- HYPERPARAMETERS CONFIGURATION FOR HIGH-PERFORMANCE ---
CONFIG = {
    "pop_size": 500,           # Large population for thorough exploration
    "n_gen": 2500,            # Extensive generational budget
    "n_neighbors": 50,         # Larger neighborhood for diversity
    "prob_mating": 0.85,       # Slightly reduced for more exploration
    "ml_global_candidates": 100000,  # More candidates for global surrogate
    "ml_local_candidates": 50000,    # Candidates for local refinement
    "polishing_iter": 100,     # More iterations for polishing
    "save_history_freq": 10,   # Save history every 10 generations
    "checkpoint_freq": 100,    # Save checkpoint every 100 generations
}

print(f"=== INITIALIZING HIGH-PERFORMANCE OPTIMIZATION PIPELINE ===")
print(f"Configuration:")
print(f"  • Generations: {CONFIG['n_gen']}")
print(f"  • Population: {CONFIG['pop_size']}")
print(f"  • Cores: {N_CORES}")
print(f"  • Global Surrogate Candidates: {CONFIG['ml_global_candidates']:,}")
print(f"  • Local Surrogate Candidates: {CONFIG['ml_local_candidates']:,}")

total_start_time = time.time()
problem = SpOCConstrainedWrapper()

# 1. MOEA/D Initialization with Large Reference Directions
print("\n[PHASE 1] Initializing MOEA/D with large population...")
ref_dirs = get_reference_directions("das-dennis", 2, n_partitions=CONFIG['pop_size']-1)

algorithm = MOEAD(
    ref_dirs,
    n_neighbors=CONFIG['n_neighbors'],
    prob_neighbor_mating=CONFIG['prob_mating'],
    seed=42,
    verbose=True
)

termination = get_termination("n_gen", CONFIG['n_gen'])

# --- PHASE 1: EXTENSIVE GLOBAL EVOLUTIONARY SEARCH ---
phase1_start = time.time()
print(f"\n[PHASE 1] Executing MOEA/D for {CONFIG['n_gen']} generations...")
print(f"  • Expected evaluations: {CONFIG['pop_size'] * CONFIG['n_gen']:,}")
print(f"  • Estimated time: {CONFIG['pop_size'] * CONFIG['n_gen'] * 0.1 / 3600:.1f} hours (assuming 0.1s/eval)")

res = minimize(
    problem, 
    algorithm, 
    termination, 
    seed=42,
    save_history=True,
    verbose=True,
    callback=None  # Can add custom callback for checkpointing
)

phase1_time = time.time() - phase1_start
print(f"\n > MOEA/D Completed in {phase1_time/3600:.2f} hours")
print(f" > Evaluations performed: {CONFIG['pop_size'] * CONFIG['n_gen']:,}")
print(f" > Solutions in archive: {len(res.X)}")

# --- PHASE 2a: GLOBAL SURROGATE-AUGMENTED SEARCH ---
phase2a_start = time.time()
print(f"\n[PHASE 2a] Global Surrogate-Augmented Search...")
print(f"  • Using {len(res.X)} evaluated solutions as training data")
print(f"  • Generating {CONFIG['ml_global_candidates']:,} virtual candidates")

ml_solutions_global = train_and_predict_surrogate_global(
    res.X, 
    res.F, 
    problem, 
    n_candidates=CONFIG['ml_global_candidates']
)

phase2a_time = time.time() - phase2a_start
print(f" > Global surrogate completed in {phase2a_time:.1f} seconds")
print(f" > Found {len(ml_solutions_global)} new valid solutions")

# --- Find current best solution for local refinement ---
print(f"\n[INTERMEDIATE] Finding current best solution...")
esa_inst = udp.constellation_udp()
current_best = None
current_best_score = float('inf')

# Check MOEA/D solutions
for i, sol in enumerate(res.X):
    x_check = list(sol.copy())
    for k in range(10, 20):
        x_check[k] = int(round(x_check[k]))
    
    try:
        fit = esa_inst.fitness(x_check)
        if fit[2] <= 0 and fit[3] <= 0:
            score = 0.97 * fit[0] + 0.03 * fit[1]
            if score < current_best_score:
                current_best_score = score
                current_best = sol.copy()
    except:
        continue

# Check global surrogate solutions
for sol in ml_solutions_global:
    x_check = list(sol.copy())
    for k in range(10, 20):
        x_check[k] = int(round(x_check[k]))
    
    try:
        fit = esa_inst.fitness(x_check)
        if fit[2] <= 0 and fit[3] <= 0:
            score = 0.97 * fit[0] + 0.03 * fit[1]
            if score < current_best_score:
                current_best_score = score
                current_best = sol.copy()
    except:
        continue

if current_best is not None:
    print(f" > Current best score: {current_best_score:.6f}")
else:
    print(" ⚠️ No valid solutions found yet. Using best from MOEA/D.")
    # Use the least infeasible solution
    best_idx = np.argmin(res.F.sum(axis=1))
    current_best = res.X[best_idx]

# --- PHASE 2b: LOCAL SURROGATE REFINEMENT (Prof's Suggestion) ---
phase2b_start = time.time()
print(f"\n[PHASE 2b] Local Surrogate Refinement (Fast neighborhood search)...")
print(f"  • Focusing on neighborhood of best solution")
print(f"  • Reusing {len(res.X)} already-evaluated points")
print(f"  • Generating {CONFIG['ml_local_candidates']:,} focused candidates")

ml_solutions_local = train_and_predict_surrogate_local(
    current_best,
    res.X,
    res.F,
    problem,
    n_candidates=CONFIG['ml_local_candidates']
)

phase2b_time = time.time() - phase2b_start
print(f" > Local surrogate completed in {phase2b_time:.1f} seconds")
print(f" > Found {len(ml_solutions_local)} refined solutions")

# --- Combine all portfolios ---
print(f"\n[COMBINING PORTFOLIOS]")
full_portfolio = list(res.X) + ml_solutions_global + ml_solutions_local
print(f"  • MOEA/D solutions: {len(res.X)}")
print(f"  • Global surrogate: {len(ml_solutions_global)}")
print(f"  • Local refinement: {len(ml_solutions_local)}")
print(f"  • Total portfolio: {len(full_portfolio)}")

# --- PHASE 3: SOLUTION SELECTION & REFINEMENT ---
phase3_start = time.time()
print(f"\n[PHASE 3] Final Selection and Polishing...")

best_score = float('inf')
best_candidate = None
valid_solutions_count = 0

print(f" > Evaluating {len(full_portfolio)} candidates for final selection...")

for i, sol in enumerate(full_portfolio):
    x_check = list(sol.copy())
    for k in range(10, 20):
        x_check[k] = int(round(x_check[k]))
    
    try:
        fit = esa_inst.fitness(x_check)
        is_valid = (fit[2] <= 0 and fit[3] <= 0)
        
        if is_valid:
            valid_solutions_count += 1
            score = 0.97 * fit[0] + 0.03 * fit[1]
            if score < best_score:
                best_score = score
                best_candidate = sol.copy()
    except:
        continue
    
    # Progress indicator
    if (i + 1) % 1000 == 0:
        print(f"   Processed {i + 1}/{len(full_portfolio)} candidates...")

if best_candidate is None:
    print("⚠️ CRITICAL: No feasible solutions in portfolio.")
    print(" > Using least infeasible solution...")
    # Find solution with smallest constraint violation
    min_violation = float('inf')
    for sol in full_portfolio:
        x_check = list(sol.copy())
        for k in range(10, 20):
            x_check[k] = int(round(x_check[k]))
        try:
            fit = esa_inst.fitness(x_check)
            violation = max(0, fit[2]) + max(0, fit[3])
            if violation < min_violation:
                min_violation = violation
                best_candidate = sol.copy()
                best_score = 0.97 * fit[0] + 0.03 * fit[1]
        except:
            continue

print(f" > Found {valid_solutions_count} valid solutions")
print(f" > Best candidate score before polishing: {best_score:.6f}")

# Apply Enhanced Local Search
print(" > Applying Enhanced Local Search (Multi-start)...")
final_solution = refine_solution_local_search(
    best_candidate, 
    esa_inst, 
    max_iter=CONFIG['polishing_iter']
)

phase3_time = time.time() - phase3_start
print(f" > Final selection completed in {phase3_time:.1f} seconds")

# --- FINAL EVALUATION ---
x_final_eval = list(final_solution.copy())
for k in range(10, 20):
    x_final_eval[k] = int(round(x_final_eval[k]))

final_stats = esa_inst.fitness(x_final_eval)
final_score = 0.97 * final_stats[0] + 0.03 * final_stats[1]
is_valid = final_stats[2] <= 0 and final_stats[3] <= 0

total_time = time.time() - total_start_time

# --- COMPREHENSIVE REPORT ---
print("\n" + "="*80)
print("🏆 HIGH-PERFORMANCE OPTIMIZATION REPORT")
print("="*80)
print(f"Total Time: {total_time/3600:.2f} hours")
print(f"Overall Status: {'VALID' if is_valid else 'INVALID'}")
print(f"Final Composite Score: {final_score:.8f} (Target: Minimize)")
print("-" * 80)
print("PHASE BREAKDOWN:")
print(f"  Phase 1 (MOEA/D):         {phase1_time/3600:7.2f} hours")
print(f"  Phase 2a (Global ML):     {phase2a_time:7.1f} seconds")
print(f"  Phase 2b (Local ML):      {phase2b_time:7.1f} seconds")
print(f"  Phase 3 (Selection/Polish): {phase3_time:7.1f} seconds")
print("-" * 80)
print("PERFORMANCE METRICS:")
print(f"  Evaluations (Real):       {CONFIG['pop_size'] * CONFIG['n_gen']:,}")
print(f"  Evaluations (Virtual):    {CONFIG['ml_global_candidates'] + CONFIG['ml_local_candidates']:,}")
print(f"  Valid Solutions Found:    {valid_solutions_count}")
print(f"  Speedup from Surrogate:   {(CONFIG['ml_global_candidates'] + CONFIG['ml_local_candidates']) * 0.1 / (phase2a_time + phase2b_time):.1f}x")
print("-" * 80)
print("FINAL SOLUTION OBJECTIVES:")
print(f"  J1 (Communication Latency): {final_stats[0]:.8f}")
print(f"  J2 (Infrastructure Cost):   {final_stats[1]:.8f}")
print("-" * 80)
print("CONSTRAINTS (<=0 is feasible):")
print(f"  C1 (Inter-Rover Distance): {final_stats[2]:.6f} km")
print(f"  C2 (Sat-Collision Safety): {final_stats[3]:.6f} km")
print("="*80)

# Export final solution
final_sol = final_solution

# =============================================================================
# 📊 ENHANCED DATA SAVING WITH CHECKPOINTS
# =============================================================================

print("\n" + "="*80)
print("💾 SAVING COMPREHENSIVE RESULTS")
print("="*80)

# Collect ALL evaluated solutions
all_solutions_X = []
all_solutions_Y = []
all_solutions_metadata = []

# 1. Solutions from MOEA/D history (sampled to reduce size)
if hasattr(res, 'history') and len(res.history) > 0:
    print(f" > Processing MOEA/D history ({len(res.history)} generations)...")
    # Sample every 10th generation to reduce file size
    for gen_idx, algo_snapshot in enumerate(res.history[::10]):
        if hasattr(algo_snapshot, 'pop') and algo_snapshot.pop is not None:
            pop = algo_snapshot.pop
            for ind in pop[::5]:  # Sample population
                x_eval = ind.X.copy()
                x_eval[10:20] = np.round(x_eval[10:20]).astype(int)
                try:
                    fit_raw = esa_inst.fitness(list(x_eval))
                    all_solutions_X.append(x_eval)
                    all_solutions_Y.append(fit_raw)
                    all_solutions_metadata.append([gen_idx * 10, 0])  # gen, source
                except:
                    pass

# 2. Add global surrogate solutions
print(f" > Adding {len(ml_solutions_global)} global surrogate solutions...")
for sol in ml_solutions_global:
    x_eval = sol.copy()
    x_eval[10:20] = np.round(x_eval[10:20]).astype(int)
    try:
        fit_raw = esa_inst.fitness(list(x_eval))
        all_solutions_X.append(x_eval)
        all_solutions_Y.append(fit_raw)
        all_solutions_metadata.append([-1, 1])  # gen=-1, source=global
    except:
        pass

# 3. Add local refined solutions
print(f" > Adding {len(ml_solutions_local)} local refined solutions...")
for sol in ml_solutions_local:
    x_eval = sol.copy()
    x_eval[10:20] = np.round(x_eval[10:20]).astype(int)
    try:
        fit_raw = esa_inst.fitness(list(x_eval))
        all_solutions_X.append(x_eval)
        all_solutions_Y.append(fit_raw)
        all_solutions_metadata.append([-2, 2])  # gen=-2, source=local
    except:
        pass

# 4. Add final polished solution
all_solutions_X.append(final_solution)
all_solutions_Y.append(final_stats)
all_solutions_metadata.append([-3, 3])  # gen=-3, source=final

# Convert to numpy arrays
xs = np.array(all_solutions_X)
ys = np.array(all_solutions_Y)
meta = np.array(all_solutions_metadata)

print(f"✓ Collected {len(xs)} solutions total")

# Deduplication with tolerance
print(" > Removing duplicates...")
unique_indices = []
seen = set()
tolerance = 1e-4

for i, x in enumerate(xs):
    # Round to tolerance for comparison
    key = tuple(np.round(x / tolerance) * tolerance)
    if key not in seen:
        seen.add(key)
        unique_indices.append(i)

xs = xs[unique_indices]
ys = ys[unique_indices]
meta = meta[unique_indices]

print(f"✓ After deduplication: {len(xs)} unique solutions")

# Create timestamp for filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'quantcomm_hpc_{timestamp}.npz'

# Save comprehensive data
np.savez_compressed(
    filename,
    xs=xs,                    # Decision variables
    ys=ys,                    # [J1, J2, c1, c2]
    metadata=meta,            # [generation, source]
    config=CONFIG,            # Configuration used
    final_solution=final_solution,
    final_score=final_score,
    total_time=total_time,
    n_cores=N_CORES
)

file_size = os.path.getsize(filename) / (1024**2)  # MB
print(f"✅ Results saved to: {filename}")
print(f"📊 File size: {file_size:.2f} MB")
print("\n📁 File contents:")
print(f"  • xs: {xs.shape} - Decision variables")
print(f"  • ys: {ys.shape} - [J1, J2, c1, c2]")
print(f"  • metadata: {meta.shape} - [generation, source]")
print(f"     - Source codes: 0=MOEA/D, 1=Global, 2=Local, 3=Final")
print(f"  • config: Hyperparameters used")
print(f"  • final_solution: Best solution found")
print(f"  • final_score: {final_score:.8f}")
print(f"  • total_time: {total_time/3600:.2f} hours")
print("="*80)

# Also save a lightweight version with only Pareto front
print("\n💡 Creating lightweight Pareto front file...")

# Extract Pareto front from valid solutions
valid_mask = (ys[:, 2] <= 0) & (ys[:, 3] <= 0)
if np.any(valid_mask):
    valid_xs = xs[valid_mask]
    valid_ys = ys[valid_mask]
    
    # Simple Pareto filter
    pareto_mask = np.ones(len(valid_xs), dtype=bool)
    for i in range(len(valid_xs)):
        for j in range(len(valid_xs)):
            if i != j and pareto_mask[i]:
                if (valid_ys[j, 0] <= valid_ys[i, 0] and 
                    valid_ys[j, 1] <= valid_ys[i, 1] and
                    (valid_ys[j, 0] < valid_ys[i, 0] or valid_ys[j, 1] < valid_ys[i, 1])):
                    pareto_mask[i] = False
                    break
    
    pareto_xs = valid_xs[pareto_mask]
    pareto_ys = valid_ys[pareto_mask]
    
    pareto_filename = f'quantcomm_pareto_{timestamp}.npz'
    np.savez_compressed(
        pareto_filename,
        xs=pareto_xs,
        ys=pareto_ys
    )
    print(f"✅ Pareto front saved to: {pareto_filename}")
    print(f"   Contains {len(pareto_xs)} non-dominated solutions")
else:
    print("⚠️ No valid solutions for Pareto front")

print("\n🎉 Optimization completed successfully!")
print(f"📈 Visualize results with:")
print(f"   data = np.load('{filename}')")
print(f"   xs = data['xs']")
print(f"   ys = data['ys']")
print("="*80)

In [ ]:
# =============================================================================
# 📊 ADVANCED ESA SCORE ANALYSIS WITH PROGRESS TRACKING
# =============================================================================
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pymoo.indicators.hv import HV
import seaborn as sns

# Set professional style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# ESA Scoring Parameters
ESA_REF_POINT = np.array([1.2, 1.4])
ESA_MULTIPLIER = 10000.0

print(f"\n" + "="*80)
print("📊 ADVANCED ESA SCORE ANALYSIS")
print("="*80)
print(f"⚙️ ESA Rules Configuration:")
print(f"  • Reference Point: {ESA_REF_POINT}")
print(f"  • Multiplier: x {ESA_MULTIPLIER:,.0f}")
print(f"  • Requirement: J1 < {ESA_REF_POINT[0]}, J2 < {ESA_REF_POINT[1]}")
print("-" * 80)

# Initialize HV calculator
ind_esa = HV(ref_point=ESA_REF_POINT)

def get_esa_score_advanced(F_pop, return_details=False):
    """
    Advanced ESA score calculation with detailed diagnostics.
    """
    if F_pop is None or len(F_pop) == 0:
        return (0.0, 0, 0) if return_details else 0.0
    
    # Filter 1: Remove physically invalid (high penalty)
    valid_physics = np.all(F_pop < 100, axis=1)
    n_physics_valid = np.sum(valid_physics)
    
    # Filter 2: ESA bounds (J1 < 1.2 AND J2 < 1.4)
    within_bounds = (F_pop[:, 0] < ESA_REF_POINT[0]) & (F_pop[:, 1] < ESA_REF_POINT[1])
    n_within_bounds = np.sum(within_bounds)
    
    # Final mask
    final_mask = valid_physics & within_bounds
    n_final = np.sum(final_mask)
    
    if n_final == 0:
        if return_details:
            return 0.0, n_physics_valid, n_within_bounds
        return 0.0
    
    # Calculate HV
    hv_raw = ind_esa(F_pop[final_mask])
    score = hv_raw * ESA_MULTIPLIER
    
    if return_details:
        return score, n_physics_valid, n_within_bounds
    return score

# Process MOEA/D history
scores = []
gens = []
valid_counts = []
bound_counts = []

if 'res' in globals() and hasattr(res, 'history') and len(res.history) > 0:
    print(f"🔄 Processing {len(res.history)} generations from MOEA/D history...")
    
    # Sample for efficiency (every 10 generations)
    sample_rate = max(1, len(res.history) // 250)  # Show ~250 points max
    
    for i, algo in enumerate(res.history[::sample_rate]):
        gen = i * sample_rate + 1
        F_pop = algo.opt.get("F")
        
        if F_pop is not None and len(F_pop) > 0:
            score, n_valid, n_bounds = get_esa_score_advanced(F_pop, return_details=True)
            scores.append(score)
            gens.append(gen)
            valid_counts.append(n_valid)
            bound_counts.append(n_bounds)
    
    if len(scores) > 0:
        final_val = scores[-1]
        print(f"✓ Processed {len(scores)} sampled generations")
    else:
        final_val = 0.0
        print("⚠️ No valid scores calculated from history")
else:
    print("⚠️ No MOEA/D history available. Using final population.")
    if 'res' in globals() and res.F is not None:
        final_val, n_valid, n_bounds = get_esa_score_advanced(res.F, return_details=True)
        scores = [final_val]
        gens = [CONFIG['n_gen']]
        valid_counts = [n_valid]
        bound_counts = [n_bounds]
    else:
        final_val = 0.0
        scores = [0.0]
        gens = [1]
        valid_counts = [0]
        bound_counts = [0]

# Create comprehensive figure
fig = plt.figure(figsize=(16, 10), dpi=150)

# 1. ESA Score Evolution
ax1 = plt.subplot(2, 2, 1)
ax1.plot(gens, scores, color='#2E86AB', linewidth=2.5, label='ESA Score')
ax1.fill_between(gens, scores, color='#2E86AB', alpha=0.2)

# Highlight final value
if len(scores) > 0:
    ax1.scatter(gens[-1], final_val, color='#A23B72', s=150, zorder=5, 
               edgecolor='black', linewidth=1.5)
    ax1.annotate(f'{final_val:,.0f}', 
                xy=(gens[-1], final_val),
                xytext=(gens[-1], final_val * 1.1),
                fontsize=11, fontweight='bold', color='#A23B72',
                ha='center', va='bottom',
                bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.9))

ax1.set_title("ESA Hypervolume Score Evolution", fontsize=14, fontweight='bold', pad=15)
ax1.set_xlabel("Generation", fontsize=12)
ax1.set_ylabel("Score (×10,000)", fontsize=12)
ax1.grid(True, linestyle='--', alpha=0.7)
ax1.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
ax1.legend(loc='lower right')

# 2. Solution Quality Metrics
ax2 = plt.subplot(2, 2, 2)
width = 0.35
x = np.arange(len(gens))

# Plot stacked bars for last 20 points or all if less
n_show = min(20, len(gens))
show_idx = slice(-n_show, None) if len(gens) > n_show else slice(None)

ax2.bar(x[show_idx], valid_counts[show_idx], width, label='Physically Valid', color='#4CB963')
ax2.bar(x[show_idx], bound_counts[show_idx], width, label='Within ESA Bounds', color='#FF9B42',
       bottom=valid_counts[show_idx])

ax2.set_title("Solution Quality Metrics", fontsize=14, fontweight='bold', pad=15)
ax2.set_xlabel("Generation (last 20 shown)", fontsize=12)
ax2.set_ylabel("Number of Solutions", fontsize=12)
ax2.legend(loc='upper left')
ax2.grid(True, axis='y', linestyle='--', alpha=0.7)

# 3. Final Population Analysis (if available)
ax3 = plt.subplot(2, 2, 3)
if 'res' in globals() and res.F is not None:
    # Extract real objectives (without penalty)
    real_F = []
    for i, sol in enumerate(res.X):
        x_check = list(sol.copy())
        for k in range(10, 20):
            x_check[k] = int(round(x_check[k]))
        try:
            fit = esa_inst.fitness(x_check)
            if fit[2] <= 0 and fit[3] <= 0:  # Only valid solutions
                real_F.append([fit[0], fit[1]])
        except:
            continue
    
    if len(real_F) > 0:
        real_F = np.array(real_F)
        scatter = ax3.scatter(real_F[:, 0], real_F[:, 1], 
                             c=0.97*real_F[:, 0] + 0.03*real_F[:, 1],
                             cmap='viridis', s=50, alpha=0.7, edgecolors='black', linewidth=0.5)
        
        # Plot ESA reference box
        ax3.axvline(x=1.2, color='red', linestyle='--', alpha=0.7, label='J1 bound')
        ax3.axhline(y=1.4, color='red', linestyle='--', alpha=0.7, label='J2 bound')
        ax3.fill_betweenx([0, 1.4], 0, 1.2, color='red', alpha=0.1)
        
        # Plot Pareto front approximation
        if len(real_F) > 1:
            # Simple Pareto front extraction
            pareto_mask = np.ones(len(real_F), dtype=bool)
            for i in range(len(real_F)):
                for j in range(len(real_F)):
                    if i != j and pareto_mask[i]:
                        if (real_F[j, 0] <= real_F[i, 0] and 
                            real_F[j, 1] <= real_F[i, 1] and
                            (real_F[j, 0] < real_F[i, 0] or real_F[j, 1] < real_F[i, 1])):
                            pareto_mask[i] = False
                            break
            
            pareto_front = real_F[pareto_mask]
            pareto_front = pareto_front[np.argsort(pareto_front[:, 0])]
            ax3.plot(pareto_front[:, 0], pareto_front[:, 1], 
                    color='#A23B72', linewidth=2.5, marker='o', 
                    markersize=8, label='Pareto Front')
        
        plt.colorbar(scatter, ax=ax3, label='Weighted Score (0.97J1+0.03J2)')
        
        ax3.set_title("Final Population (Valid Solutions)", fontsize=14, fontweight='bold', pad=15)
        ax3.set_xlabel("J1 (Communication Latency)", fontsize=12)
        ax3.set_ylabel("J2 (Infrastructure Cost)", fontsize=12)
        ax3.legend(loc='upper right')
        ax3.grid(True, linestyle='--', alpha=0.7)
        ax3.set_xlim([0, max(2.0, np.max(real_F[:, 0]) * 1.1)])
        ax3.set_ylim([0, max(2.0, np.max(real_F[:, 1]) * 1.1)])
    else:
        ax3.text(0.5, 0.5, 'No Valid Solutions\nin Final Population', 
                ha='center', va='center', transform=ax3.transAxes, fontsize=12)
        ax3.set_title("Final Population Analysis", fontsize=14, fontweight='bold', pad=15)
else:
    ax3.text(0.5, 0.5, 'No Result Data Available', 
            ha='center', va='center', transform=ax3.transAxes, fontsize=12)
    ax3.set_title("Final Population Analysis", fontsize=14, fontweight='bold', pad=15)

# 4. Performance Summary
ax4 = plt.subplot(2, 2, 4)
ax4.axis('off')

# Create summary text
summary_text = (
    f"OPTIMIZATION SUMMARY\n"
    f"═══════════════════════\n"
    f"• Final ESA Score: {final_val:,.0f}\n"
    f"• Total Time: {total_time/3600:.1f} hours\n"
    f"• Generations: {CONFIG['n_gen']:,}\n"
    f"• Population Size: {CONFIG['pop_size']:,}\n"
    f"• CPU Cores Used: {N_CORES}\n"
    f"• Valid Solutions: {valid_solutions_count:,}\n"
    f"• Final J1: {final_stats[0]:.6f}\n"
    f"• Final J2: {final_stats[1]:.6f}\n"
    f"• Feasibility: {'✓' if is_valid else '✗'}\n"
    f"═══════════════════════\n"
    f"SURROGATE PERFORMANCE\n"
    f"• Global Candidates: {CONFIG['ml_global_candidates']:,}\n"
    f"• Local Candidates: {CONFIG['ml_local_candidates']:,}\n"
    f"• Speedup Factor: {(CONFIG['ml_global_candidates'] + CONFIG['ml_local_candidates']) * 0.1 / (phase2a_time + phase2b_time):.1f}x"
)

ax4.text(0.1, 0.95, summary_text, fontfamily='monospace', fontsize=10,
        verticalalignment='top', linespacing=1.5,
        bbox=dict(boxstyle="round,pad=1", facecolor='lightgray', alpha=0.8))

ax4.set_title("Performance Summary", fontsize=14, fontweight='bold', pad=15)

plt.suptitle(f"ESA SpOC Challenge Optimization Results - {timestamp}", 
            fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()

# Save figure
figure_filename = f'esa_analysis_comprehensive_{timestamp}.png'
plt.savefig(figure_filename, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print(f"\n📈 Analysis completed!")
print(f"✅ Figure saved as: {figure_filename}")

# Final ESA Score Report
print("\n" + "="*80)
print("📋 FINAL ESA SCORE REPORT")
print("="*80)
print(f"Final ESA Score: {final_val:,.4f}")

if final_val == 0:
    print("\n❌ CRITICAL: Score is ZERO!")
    print("   Reasons:")
    print("   1. No solutions satisfy J1 < 1.2 AND J2 < 1.4")
    print("   2. All solutions violate physical constraints")
    print("   3. No valid solutions found in the search")
    print("\n   Recommended actions:")
    print("   • Increase population size")
    print("   • Run for more generations")
    print("   • Adjust penalty weights")
    print("   • Check constraint handling")
else:
    print("\n✅ SUCCESS: Valid ESA score achieved!")
    print(f"   • Score: {final_val:,.0f}")
    print(f"   • Solutions in ESA box: {bound_counts[-1] if len(bound_counts) > 0 else 'N/A'}")
    print(f"   • Physical validity: {valid_counts[-1] if len(valid_counts) > 0 else 'N/A'} solutions")
    
    # Quality assessment
    if final_val > 10000:
        print("   🎉 EXCELLENT: High score achieved!")
    elif final_val > 5000:
        print("   👍 GOOD: Competitive score")
    else:
        print("   ⚠️  MODERATE: Room for improvement")

print("="*80)

# Additional diagnostic if score is low
if 0 < final_val < 1000:
    print("\n🔍 DIAGNOSTIC ANALYSIS:")
    print("   Low score indicates:")
    print("   1. Few solutions within ESA bounds")
    print("   2. Limited hypervolume coverage")
    print("   3. Possible constraint violations")
    print("\n   Suggestions:")
    print("   1. Focus search near [J1=1.0, J2=1.2] region")
    print("   2. Increase local surrogate refinement")
    print("   3. Adjust weighted sum coefficients")

print("\n" + "="*80)
print("🎯 OPTIMIZATION COMPLETED SUCCESSFULLY")
print("="*80)